In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random

import numpy as np
import pandas as pd

from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0
from src.models.gcn import GCN1
from src.models.gat import GAT
from notebooks.FinalAnalysisAbdul.models import GCN

from src.models.train_model import train_with_loss
from src.models.train_model import random_splits

from src.models.reg import make_preg_loss
from src.models.reg import make_lap_loss
from src.models.reg import make_preg_ce_ce

from src.models.reg import compute_a_hat

from src.models.evaluate_model import evaluate0
from src.models.evaluate_model import evaluate1
from src.models.evaluate_model import evaluate1abdul

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from sklearn.manifold import TSNE

from src.models.evaluate_model import test
from notebooks.FinalAnalysisAbdul.p_reg_loss import A_hat_computations

ATTENTION!!! MODIFIED P TO Z IN SOME PARTS OF MAKE_PREG_LOSS!!!


In [3]:
# set up torch env and dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=T.NormalizeFeatures())
data = dataset[0].to(device)

In [4]:
seed = 0
A_hat, A_hat_mask, N = A_hat_computations(data)

metrics = []
for mu in range(11):
    torch.manual_seed(seed)
    random.seed(seed)
    
    mu = mu / 10
    loss_fn = make_preg_ce_ce(mu, A_hat)
        
    model = GCN1(dataset.num_node_features, dataset.num_classes)

    model = train_with_loss(model, data, loss_fn, lr=0.01, weight_decay=5e-4, num_epochs=400, beta=0.01)

    train_acc, val_acc, test_acc = evaluate1(model, data)
    metrics.append({'mu': mu, 'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc})
    print(metrics[-1])



{'mu': 0.0, 'train_acc': 0.9571428571428572, 'val_acc': 0.942, 'test_acc': 0.943}
